## Create dataset for matchday

In [17]:
import csv
import pandas as pd

In [18]:
input = '/content/drive/MyDrive/Inframind/Customer engagement/original_dataset/processed_dataset.csv'
df = pd.read_csv(input)
df.tail()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,match_id,inning,batting_team,bowling_team,ball,batsman,bowler,batsman_runs,player_dismissed,is_wicket,wickets_so_far,last_3_overs_runs,last_6_balls_runs,last_3_overs_wickets,last_6_balls_wickets,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,wicket_next_6_balls,wicket_next_12_balls,boundary_next_6_balls,boundary_next_12_balls,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate
714709,1596,2,South Africa,England,46.4,CH Morris,BA Stokes,0,NaN,0,8,41,11,0,0,31,250,9.8,0,1,1,0,0,0,0,100.43,41.71,20.3,41.6
714710,1596,2,South Africa,England,46.5,CH Morris,BA Stokes,1,NaN,0,8,35,7,0,0,32,250,9.4,1,1,1,0,0,0,0,100.43,41.71,20.3,41.6
714711,1596,2,South Africa,England,46.6,KJ Abbott,BA Stokes,0,NaN,0,8,35,7,0,0,33,251,9.6,2,1,1,0,0,0,0,60.31,41.71,8.44,41.6
714712,1596,2,South Africa,England,47.1,CH Morris,AU Rashid,0,CH Morris,1,8,35,7,0,0,34,251,9.6,3,1,1,0,0,0,1,100.43,31.79,20.3,33.9
714713,1596,2,South Africa,England,47.2,Imran Tahir,AU Rashid,4,NaN,0,9,31,7,1,1,0,251,9.6,4,0,0,0,0,0,0,NaN,NaN,NaN,NaN


## Generate innings based feature

In [19]:
# add column required_run_rate
target = 0
required_run_rate = []
for i in df.index:

    if (df['inning'][i] == 2):
      if (df['ball'][i] == 0.1):
        target = df['runs_so_far'][i-1]
        required_run_rate.append( (target - df['runs_so_far'][i]) / (50.00 - df['ball'][i]) )
      else:
        required_run_rate.append( (target - df['runs_so_far'][i]) / (50.00 - df['ball'][i]) )

    else:
      # inning 1
      required_run_rate.append(0)      

df['required_run_rate'] = required_run_rate

df.head()

,match_id,inning,batting_team,bowling_team,ball,batsman,bowler,batsman_runs,player_dismissed,is_wicket,wickets_so_far,last_3_overs_runs,last_6_balls_runs,last_3_overs_wickets,last_6_balls_wickets,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,wicket_next_6_balls,wicket_next_12_balls,boundary_next_6_balls,boundary_next_12_balls,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate
0,1,1,Australia,Pakistan,0.1,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,0,0,0.0,0,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0
1,1,1,Australia,Pakistan,0.2,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,1,0,0.0,1,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0
2,1,1,Australia,Pakistan,0.3,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,2,0,0.0,2,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0
3,1,1,Australia,Pakistan,0.4,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,3,0,0.0,3,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0
4,1,1,Australia,Pakistan,0.5,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,4,0,0.0,4,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0


## Data Filtration

In [20]:
# filter and clean bowling_average 
for i in df.index:
  if df['bowling_average'][i] == '-':
    df.at[i, 'bowling_average'] = 40.00

df["bowling_average"].fillna(40.00, inplace = True) 


In [21]:
# filter and clean batting_average 
for i in df.index:
  if df['batting_average'][i] == '-':
    df.at[i, 'batting_average'] = 28.00

df["batting_average"].fillna(20.00, inplace = True) 


In [22]:
# filter and clean bowling_strike_rate 
for i in df.index:
  if df['bowling_strike_rate'][i] == '-':
    df.at[i, 'bowling_strike_rate'] = 70.00

df["bowling_strike_rate"].fillna(70.00, inplace = True) 


In [23]:
# filter and clean strike_rate 
for i in df.index:
  if df['strike_rate'][i] == '-':
    df.at[i, 'strike_rate'] = 60.00

df["strike_rate"].fillna(60.00, inplace = True) 


In [24]:
# one hot encoding of col "batting_team"

# Get one hot encoding of col "batting team"
one_hot = pd.get_dummies(df['bowling_team'], prefix='bowling_team')
# Drop that column as it is now encoded
df = df.drop('bowling_team',axis = 1)
# Join the encoded one with df
df = df.join(one_hot)

df.head()

,match_id,inning,batting_team,ball,batsman,bowler,batsman_runs,player_dismissed,is_wicket,wickets_so_far,last_3_overs_runs,last_6_balls_runs,last_3_overs_wickets,last_6_balls_wickets,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,wicket_next_6_balls,wicket_next_12_balls,boundary_next_6_balls,boundary_next_12_balls,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate,bowling_team_Afghanistan,bowling_team_Australia,bowling_team_Bangladesh,bowling_team_England,bowling_team_India,bowling_team_New Zealand,bowling_team_Pakistan,bowling_team_South Africa,bowling_team_Sri Lanka,bowling_team_West Indies
0,1,1,Australia,0.1,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,0,0,0.0,0,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0
1,1,1,Australia,0.2,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,1,0,0.0,1,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0
2,1,1,Australia,0.3,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,2,0,0.0,2,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0
3,1,1,Australia,0.4,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,3,0,0.0,3,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0
4,1,1,Australia,0.5,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,4,0,0.0,4,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0


## One-hot encode categorical features

In [25]:
# one hot encoding of col "batting_team"

# Get one hot encoding of col "batting team"
one_hot = pd.get_dummies(df['batting_team'], prefix='batting_team')
# Drop that column as it is now encoded
df = df.drop('batting_team',axis = 1)
# Join the encoded one with df
df = df.join(one_hot)

df.head()

,match_id,inning,ball,batsman,bowler,batsman_runs,player_dismissed,is_wicket,wickets_so_far,last_3_overs_runs,last_6_balls_runs,last_3_overs_wickets,last_6_balls_wickets,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,wicket_next_6_balls,wicket_next_12_balls,boundary_next_6_balls,boundary_next_12_balls,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate,bowling_team_Afghanistan,bowling_team_Australia,bowling_team_Bangladesh,bowling_team_England,bowling_team_India,bowling_team_New Zealand,bowling_team_Pakistan,bowling_team_South Africa,bowling_team_Sri Lanka,bowling_team_West Indies,batting_team_Afghanistan,batting_team_Australia,batting_team_Bangladesh,batting_team_England,batting_team_India,batting_team_New Zealand,batting_team_Pakistan,batting_team_South Africa,batting_team_Sri Lanka,batting_team_West Indies
0,1,1,0.1,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,0,0,0.0,0,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,1,0.2,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,1,0,0.0,1,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,1,0.3,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,2,0,0.0,2,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1,1,0.4,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,3,0,0.0,3,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1,1,0.5,DA Warner,Mohammad Amir,0,NaN,0,0,14,3,0,0,4,0,0.0,4,0,0,0,0,0,0,95.94,29.62,45.77,37.1,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [26]:
df['strike_rate'] = df['strike_rate'].astype(float)
df['bowling_average'] = df['bowling_average'].astype(float)
df['batting_average'] = df['batting_average'].astype(float)
df['bowling_strike_rate'] = df['bowling_strike_rate'].astype(float)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714714 entries, 0 to 714713
Data columns (total 48 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   match_id                   714714 non-null  int64  
 1   inning                     714714 non-null  int64  
 2   ball                       714714 non-null  float64
 3   batsman                    714714 non-null  object 
 4   bowler                     714714 non-null  object 
 5   batsman_runs               714714 non-null  int64  
 6   player_dismissed           19074 non-null   object 
 7   is_wicket                  714714 non-null  int64  
 8   wickets_so_far             714714 non-null  int64  
 9   last_3_overs_runs          714714 non-null  int64  
 10  last_6_balls_runs          714714 non-null  int64  
 11  last_3_overs_wickets       714714 non-null  int64  
 12  last_6_balls_wickets       714714 non-null  int64  
 13  balls_in_partnership       71

## Drop irrelevant features

In [28]:
df.drop(['bowling_team_Afghanistan', 'bowling_team_Australia',
       'bowling_team_Bangladesh', 'bowling_team_England', 'bowling_team_India',
       'bowling_team_New Zealand', 'bowling_team_Pakistan',
       'bowling_team_South Africa', 'bowling_team_Sri Lanka',
       'bowling_team_West Indies'], axis=1, inplace=True)


In [29]:
df.drop(['batting_team_Afghanistan', 'batting_team_Australia',
       'batting_team_Bangladesh', 'batting_team_England', 'batting_team_India',
       'batting_team_New Zealand', 'batting_team_Pakistan',
       'batting_team_South Africa', 'batting_team_Sri Lanka',
       'batting_team_West Indies'], axis=1, inplace=True)


In [30]:
# filter by match id 
df = df[df['match_id'] == 1431]

In [31]:
df.drop(['match_id', 'batsman', 'bowler',
        'batsman_runs', 'wicket_next_6_balls','boundary_next_6_balls' , 'player_dismissed', 'boundary_next_12_balls','wicket_next_12_balls',
         'is_wicket'], axis = 1, inplace = True) 
df.head()

,inning,ball,wickets_so_far,last_3_overs_runs,last_6_balls_runs,last_3_overs_wickets,last_6_balls_wickets,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate
583458,1,0.1,0,14,3,0,0,0,0,0.000000,0,1,0,87.34,34.6,42.20,41.2,0.0
583459,1,0.2,0,14,3,0,0,1,1,5.000000,1,1,0,95.31,34.6,33.21,41.2,0.0
583460,1,0.3,0,14,3,0,0,2,1,3.333333,2,1,0,95.31,34.6,33.21,41.2,0.0
583461,1,0.4,0,14,3,0,0,3,1,2.500000,3,1,0,95.31,34.6,33.21,41.2,0.0
583462,1,0.5,0,14,3,0,0,4,1,2.000000,4,1,0,95.31,34.6,33.21,41.2,0.0


In [32]:
# dropping more irrelvenat features
df.drop(['last_3_overs_runs', 'last_3_overs_wickets','last_6_balls_runs', 'last_6_balls_wickets'], axis=1, inplace=True)

## Generate final predictive feature

In [33]:
# make a column for exciting moment
# if either wicket_this_over or boundary_this_ovre is true, exciting_moment is true


big_moment = []
for i in df.index:
    if (df['wicket_this_over'][i] == 1 or df['boundary_this_over'][i]==1):
      big_moment.append(1)
    else:
      big_moment.append(0)     

df['big_moment'] = big_moment

df.head()

,inning,ball,wickets_so_far,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate,big_moment
583458,1,0.1,0,0,0,0.000000,0,1,0,87.34,34.6,42.20,41.2,0.0,1
583459,1,0.2,0,1,1,5.000000,1,1,0,95.31,34.6,33.21,41.2,0.0,1
583460,1,0.3,0,2,1,3.333333,2,1,0,95.31,34.6,33.21,41.2,0.0,1
583461,1,0.4,0,3,1,2.500000,3,1,0,95.31,34.6,33.21,41.2,0.0,1
583462,1,0.5,0,4,1,2.000000,4,1,0,95.31,34.6,33.21,41.2,0.0,1


## Divde and export data by ininings

In [34]:
# divide inining 1 and inning 2 data
df1 = df[df['inning'] == 1]
df2 = df[df['inning'] == 2]

In [35]:
df1

,inning,ball,wickets_so_far,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate,big_moment
583458,1,0.1,0,0,0,0.000000,0,1,0,87.34,34.60,42.20,41.2,0.0,1
583459,1,0.2,0,1,1,5.000000,1,1,0,95.31,34.60,33.21,41.2,0.0,1
583460,1,0.3,0,2,1,3.333333,2,1,0,95.31,34.60,33.21,41.2,0.0,1
583461,1,0.4,0,3,1,2.500000,3,1,0,95.31,34.60,33.21,41.2,0.0,1
583462,1,0.5,0,4,1,2.000000,4,1,0,95.31,34.60,33.21,41.2,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583759,1,49.3,7,2,279,8.400000,11,0,0,98.54,30.98,12.75,32.4,0.0,0
583760,1,49.4,7,3,279,8.200000,12,0,0,98.54,30.98,12.75,32.4,0.0,0
583761,1,49.5,7,4,280,7.200000,13,0,0,108.72,30.98,27.72,32.4,0.0,0
583762,1,49.6,7,5,282,6.800000,14,0,0,108.72,30.98,27.72,32.4,0.0,0


In [36]:
df2

,inning,ball,wickets_so_far,balls_in_partnership,runs_so_far,run_rate_last_5_overs,balls_since_last_boundary,boundary_this_over,wicket_this_over,strike_rate,bowling_average,batting_average,bowling_strike_rate,required_run_rate,big_moment
583764,2,0.1,0,0,0,0.0,0,0,0,88.92,34.26,49.14,37.7,5.671343,0
583765,2,0.2,0,1,0,0.0,1,0,0,88.92,34.26,49.14,37.7,5.682731,0
583766,2,0.3,0,2,0,0.0,2,0,0,88.92,34.26,49.14,37.7,5.694165,0
583767,2,0.4,0,3,0,0.0,3,0,0,88.92,34.26,49.14,37.7,5.705645,0
583768,2,0.5,0,4,0,0.0,4,0,0,88.92,34.26,49.14,37.7,5.717172,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584063,2,47.6,9,3,247,7.4,8,0,0,81.66,35.40,7.55,39.6,15.000000,0
584064,2,48.1,9,4,248,6.4,9,0,1,81.66,34.26,7.55,37.7,18.421053,1
584065,2,48.2,9,5,252,7.2,0,0,1,81.66,34.26,7.55,37.7,17.222222,1
584066,2,48.3,9,6,253,7.0,1,0,1,35.46,34.26,4.80,37.7,17.647059,1


In [37]:
df1.to_csv('/content/drive/MyDrive/Inframind/Customer engagement/original_dataset/match_1.csv',index=False)


In [38]:
df2.to_csv('/content/drive/MyDrive/Inframind/Customer engagement/original_dataset/match_2.csv',index=False)
